<a href="https://colab.research.google.com/github/RobBurnap/Bioinformatics-MICR4203-MICR5203/blob/main/notebooks/L01_foundations_week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# BIOINFO4/5203 — Week 1 Exercise (Foundations)

**Goals for today**
- Mount Google Drive and create your course folders
- Load a small FASTA file
- Compute simple sequence statistics
- Save a plot and a summary text into your `Outputs/` folder

> **Deliverables to Canvas:** the executed notebook (`.ipynb`) and a PDF export with outputs visible.


## A. Setup — install dependencies & mount Drive

In [1]:

# Install FIRST, then import
%pip install -q biopython

from google.colab import drive; drive.mount('/content/drive')

import os, pandas as pd
from Bio import SeqIO
import matplotlib.pyplot as plt

print("✅ Dependencies installed & Drive mounted.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 1.3 MB/s eta 0:00:00
Mounted at /content/drive
✅ Dependencies installed & Drive mounted.



## B. Course folders

Edit only `LECTURE_CODE` and `TOPIC` if needed. All inputs will live in `Data/LECTURE_TOPIC` and outputs in `Outputs/LECTURE_TOPIC`.


In [2]:

# --- Course folder config (customize LECTURE_CODE/TOPIC only) ---
COURSE_DIR   = "/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25"
LECTURE_CODE = "L01"            # change per week (e.g., L02, L03, ...)
TOPIC        = "foundations"    # short slug for the exercise

# Derived paths (do not change)
DATA_DIR   = f"{COURSE_DIR}/Data/{LECTURE_CODE}_{TOPIC}"
OUTPUT_DIR = f"{COURSE_DIR}/Outputs/{LECTURE_CODE}_{TOPIC}"

# Create folder structure if missing
for p in [f"{COURSE_DIR}/Data", f"{COURSE_DIR}/Outputs", f"{COURSE_DIR}/Notebooks", DATA_DIR, OUTPUT_DIR]:
    os.makedirs(p, exist_ok=True)

print("📁 COURSE_DIR :", COURSE_DIR)
print("📁 DATA_DIR   :", DATA_DIR)
print("📁 OUTPUT_DIR :", OUTPUT_DIR)


📁 COURSE_DIR : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25
📁 DATA_DIR   : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Data/L01_foundations
📁 OUTPUT_DIR : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L01_foundations



### Optional: create a tiny demo FASTA (runs only if none found)
If you already placed a FASTA file in `Data/`, you can skip this. Otherwise, run this cell to generate a small example.


In [3]:

# Write a tiny demo FASTA only if none exists yet
existing_fastas = [f for f in os.listdir(DATA_DIR) if f.lower().endswith((".fa", ".fasta", ".faa"))]
if not existing_fastas:
    demo_fa = f"{DATA_DIR}/demo.fasta"
    with open(demo_fa, "w") as f:
        f.write(">seqA\nMSTNPKPQRK\n>seqB\nMSSSNTATAP\n>seqC\nATGGGCCCTTTAA\n")
    print("🧪 Created demo FASTA:", demo_fa)
else:
    print("✅ FASTA already present:", existing_fastas)


🧪 Created demo FASTA: /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Data/L01_foundations/demo.fasta



## C. Student instructions
1. Put a small FASTA file into your data folder shown above (or use the demo created).  
2. Run the next cells to load, summarize, and plot your sequences.  
3. Confirm that outputs are written into your `Outputs/` folder for this week.


In [4]:

# Find a FASTA file in DATA_DIR
fasta_path = None
for fname in os.listdir(DATA_DIR):
    if fname.lower().endswith((".fa", ".fasta", ".faa")):
        fasta_path = f"{DATA_DIR}/{fname}"
        break
assert fasta_path, "No FASTA found in Data/. Add a FASTA or run the demo cell above."

# Parse sequences
records = list(SeqIO.parse(fasta_path, "fasta"))
assert len(records) > 0, "No records found in FASTA."
ids = [r.id for r in records]
lengths = [len(r.seq) for r in records]

# Compute base composition (A,C,G,T) or amino acids if protein
def base_counts(seq):
    s = str(seq).upper()
    return {
        "A": s.count("A"),
        "C": s.count("C"),
        "G": s.count("G"),
        "T": s.count("T")
    }

comps = [base_counts(r.seq) for r in records]

import pandas as pd
df = pd.DataFrame({
    "id": ids,
    "length": lengths,
    "A": [c["A"] for c in comps],
    "C": [c["C"] for c in comps],
    "G": [c["G"] for c in comps],
    "T": [c["T"] for c in comps],
})

print("🔎 Parsed:", fasta_path)
display(df)


🔎 Parsed: /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Data/L01_foundations/demo.fasta


,id,length,A,C,G,T
0,seqA,10,0,0,0,1
1,seqB,10,2,0,0,2
2,seqC,13,3,3,3,4


## D. Save outputs (CSV + PNG)

In [5]:

# Save CSV
csv_path = f"{OUTPUT_DIR}/seq_summary.csv"
df.to_csv(csv_path, index=False)
print("💾 Saved CSV ->", csv_path)

# Plot lengths
plt.figure()
df.set_index("id")["length"].plot(kind="bar")
plt.title("Sequence lengths")
plt.xlabel("Sequence ID"); plt.ylabel("Length")
png_path = f"{OUTPUT_DIR}/lengths_barplot.png"
plt.savefig(png_path, bbox_inches="tight")
plt.close()
print("🖼️ Saved PNG ->", png_path)

# Verify directory contents
print("📦 Output dir listing:", os.listdir(OUTPUT_DIR))


💾 Saved CSV -> /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L01_foundations/seq_summary.csv
🖼️ Saved PNG -> /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L01_foundations/lengths_barplot.png
📦 Output dir listing: ['seq_summary.csv', 'lengths_barplot.png']


## E. Results summary (copy into Canvas if requested)

In [6]:

summary_path = f"{OUTPUT_DIR}/summary.txt"
with open(summary_path, "w") as f:
    f.write(f"LECTURE={LECTURE_CODE}\n")
    f.write(f"TOPIC={TOPIC}\n")
    f.write(f"N_records={len(records)}\n")
    f.write(f"FASTA={os.path.basename(fasta_path)}\n")
print("📝 Saved summary ->", summary_path)

print("=== SUMMARY ===")
print("LECTURE=", LECTURE_CODE)
print("TOPIC=", TOPIC)
print("N_records=", len(records))
print("FASTA=", os.path.basename(fasta_path))


📝 Saved summary -> /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L01_foundations/summary.txt
=== SUMMARY ===
LECTURE= L01
TOPIC= foundations
N_records= 3
FASTA= demo.fasta



## F. Export & submit
- **File → Print → Save as PDF**, then upload the PDF and `.ipynb` to Canvas.  
- Ensure your `Outputs/` folder contains: `seq_summary.csv`, `lengths_barplot.png`, and `summary.txt`.
